In [1]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
import pprint
import os
from bs4 import BeautifulSoup
import requests
import json
import jsonschema
from jsonschema import validate
import re
from datetime import datetime

### Git Client connection to Azure DevOps

The code below uses connects to your Organization and uses a Personal Access Token (PAT) to authenticate to Azure DevOps  

You will need to provide:
1. the url to your organization 
2. a PAT against the organization that is listed in the URL


In [3]:
# Get a client (the "core" client provides access to projects, teams, etc)
def get_clients_projects():

    # Fill in with your personal access token and org URL
    personal_access_token = os.getenv("gitpat")
    organization_url = os.getenv("org_url")

    # Create a connection to the org
    credentials = BasicAuthentication('', personal_access_token)
    connection = Connection(base_url=organization_url, creds=credentials)    
    
    core_client = connection.clients.get_core_client()
    git_client = connection.clients.get_git_client()

    # Get the first page of projects
    get_projects_response = core_client.get_projects()
    return core_client, git_client, get_projects_response

### Simple JSON validation

This is not 100% accurrate but is sufficient for most use cases in testing whether a string is in fact a valid JSON string.

In [4]:
def json_validator(data):
    try:
        json.loads(data)
        return True
    except ValueError as error:
        print("invalid json: %s" % error)
        return False

### Reading all JSON files in the pipeline folder

The following will read all files in the pipeline folder for every repository in an organization. Further, it will modify only the specified branch in any repository that has a matching name. The ADF Activity type that is is specifically looking for is the "copy" activity. If you need to search on more then a for loop should be created to iterate through all the activities.

In [ ]:
core_client, git_client, get_projects_response = get_clients_projects()

from base64 import b64encode
from base64 import decode

branch_to_find = 'test3a'

# whole number
retryIntervalInSeconds = 50

# whole number
retry = 1

# string value formatted as Days, Hours, Minutes, Seconds {0.0:00:00}
timeout = "0:05:00"

index = 0
while get_projects_response is not None:
    for project in get_projects_response.value:
        repos = git_client.get_repositories(project.id)
        for repo in repos:
            try:
                branches = git_client.get_branches(repository_id=repo.id)
                for branch in branches:
                    if branch.name != branch_to_find:
                        print(f'These are not the droids you are looking for: {branch.name}')
                        continue

                    print(f'Am I in test3a: {"test3a" in branch.name}')
                    commit = git_client.get_commit(repository_id=repo.id, commit_id=branch.commit.commit_id)
                    tree = git_client.get_tree(repository_id=repo.id, sha1=commit.tree_id)

                    for entry in tree.tree_entries:
                        if entry.relative_path == "pipeline":
                            cc = git_client.get_tree(repository_id=repo.id, sha1=entry.object_id)
                            for c in cc.tree_entries:
                                if c.git_object_type == 'blob':
                                    blob = git_client.get_blob_content(repository_id=repo.id, sha1=c.object_id)
                                    try:
                                        data = json.loads(next(blob))
                                    except Exception as ex:
                                        print(f'failed to read json file {ex}')
                                    s = json.dumps(data)
                                    print(f'what did it look like before {s}')

                                    if s.index("\"type\": \"Copy\""):               

                                        print(f'branch name is {branch.name}')
                                        found_value = s[s.index("\"timeout\""): s.find(",", s.index("timeout"))]
                                        s = s.replace(found_value,f'"timeout": {timeout}') 
                                        found_value = s[s.index("\"retryIntervalInSeconds\""): s.find(",", s.index("retryIntervalInSeconds"))]
                                        s = s.replace(found_value,f'"retryIntervalInSeconds": {retryIntervalInSeconds}')
                                        found_value = s[s.index("\"retry\""): s.find(",", s.index("retry"))]
                                        s = s.replace(found_value,f'"retry": {retry}')

                                        try:
                                            s = json.loads(s)
                                        except Exception as mm:
                                            print(f'error on converting to dict: {mm}')
                                        
                                        # this is close but AzDO doesn't decode this and leaves it as bytes
                                        # my_json = b64encode(json.dumps(s,indent=4).encode('ascii'))
                                        # string_byte_json = str(my_json).replace("'b","").replace("'","")

                                        my_json =   json.dumps(s,indent=4)

                                        push = {"refUpdates":[{"name":"refs/heads/" + branch.name,"oldObjectId": commit.commit_id }],"commits":[{"comment":"Updating readme via python and Azure DevOps SDK: " + datetime.now().strftime("%Y %B, %A %w, %H hours %M minutes")  ,"changes":[{"changeType":"edit","item":{"path":"/pipeline/" + c.relative_path},"newContent":{"content": my_json ,"contentType":"rawText"}}]}]}

                                        git_client.create_push(
                                            repository_id=repo.id, push=push
                                        )

            except Exception as e:
                print(e)
        index += 1

        #project.
    if get_projects_response.continuation_token is not None and get_projects_response.continuation_token != "":
        # Get the next page of projects
        get_projects_response = core_client.get_projects(continuation_token=get_projects_response.continuation_token)

    else:
        # All projects have been retrieved
        get_projects_response = None